<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#reader" data-toc-modified-id="reader-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>reader</a></span><ul class="toc-item"><li><span><a href="#reader-细节解读" data-toc-modified-id="reader-细节解读-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>reader 细节解读</a></span></li><li><span><a href="#tf.strided_slice" data-toc-modified-id="tf.strided_slice-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>tf.strided_slice</a></span></li><li><span><a href="#reader-输出结果" data-toc-modified-id="reader-输出结果-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>reader 输出结果</a></span></li></ul></li><li><span><a href="#ptb-word-lm" data-toc-modified-id="ptb-word-lm-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>ptb-word-lm</a></span><ul class="toc-item"><li><span><a href="#dropout" data-toc-modified-id="dropout-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>dropout</a></span></li></ul></li></ul></div>

官方文档，学习的好例子。  
[Recurrent Neural Networks  |  TensorFlow](https://www.tensorflow.org/tutorials/recurrent)

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import os
import sys

import tensorflow as tf
import numpy as np
import random

# reader

## reader 细节解读

In [70]:
sess = tf.Session()

In [27]:
# 输入的就是 数字 id，one-hot
raw_data = np.array([random.randint(1,1000) for i in range(1000)])

In [29]:
# id 转 tensor
raw_data = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.int32)
sess.run(raw_data[:20])

array([545, 566, 656, 363, 311,  48, 885, 359, 653, 441, 558, 199, 260,
       741, 880, 263, 923, 127, 319, 667], dtype=int32)

In [30]:
# tf.size，展开后数据大小
data_len = tf.size(raw_data)
sess.run(data_len)

1000

In [35]:
# 一个 batch 有多少数据
batch_size = 8 # batch 数量，批大小
batch_len = data_len // batch_size
sess.run(batch_len)

125

In [36]:
# 每一列是一个 batch
data = tf.reshape(raw_data[0 : batch_size * batch_len], [batch_size, batch_len])
print(data.shape)
print(sess.run(batch_size * batch_len))
sess.run(data[0])

(8, ?)
1000


array([545, 566, 656, 363, 311,  48, 885, 359, 653, 441, 558, 199, 260,
       741, 880, 263, 923, 127, 319, 667, 332, 537, 898, 166, 587, 303,
       871, 454, 786, 832, 380, 376,  59, 990, 664,  78, 906, 355, 906,
       287, 568, 507, 333, 903, 273, 429, 503, 670, 900, 788, 128, 136,
       379, 138, 321, 439, 993, 609, 800, 207, 518,  98, 548, 620,  84,
       441, 602, 575, 357, 420, 833, 439, 760, 143, 371, 212, 418, 597,
        40, 210, 770, 403,  25, 208, 266, 269, 199, 992, 765, 280, 821,
       977, 306, 296, 312, 353,  44, 361, 203, 252, 341, 859, 186, 974,
       718,  54, 612, 249, 389,  17, 767, 137, 654, 271, 470, 235,  73,
       192, 625, 386, 906,  83, 166,  72,  54], dtype=int32)

In [44]:
# time_step = num_step
num_steps = 20
epoch_size = (batch_len - 1) // num_steps
sess.run(epoch_size) # 6 个 epoch 构成一个 batch， 20 * 6 = 120
# 也就是 6 个 epoch 才能把一个 batch 跑完

6

In [39]:
i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
i

<tf.Tensor 'input_producer_Dequeue:0' shape=() dtype=int32>

In [40]:
x = tf.strided_slice(data, [0, i*num_steps], [batch_size, (i+1)*num_steps])
x

<tf.Tensor 'StridedSlice:0' shape=(?, ?) dtype=int32>

In [41]:
x.set_shape([batch_size, num_steps])
x.shape

TensorShape([Dimension(8), Dimension(20)])

In [42]:
y = tf.strided_slice(data, [0, i * num_steps + 1],
                     [batch_size, (i + 1) * num_steps + 1])
y.set_shape([batch_size, num_steps])

In [43]:
y.shape

TensorShape([Dimension(8), Dimension(20)])

## tf.strided_slice

In [54]:
input = np.array([[[1, 1, 1], [2, 2, 2]],[[3, 3, 3], [4, 4, 4]], [[5, 5, 5], [6, 6, 6]]])

In [55]:
input

array([[[1, 1, 1],
        [2, 2, 2]],

       [[3, 3, 3],
        [4, 4, 4]],

       [[5, 5, 5],
        [6, 6, 6]]])

In [84]:
# 1,2 保证 [[3,3,3],[4,4,4]] 
# 0,1 保证 [3,3,3]
# 2,3 保证 [3]
sess.run(tf.strided_slice(input, [1, 0, 2], [2, 1, 3], [1, 1, 1]))

array([[[3]]])

In [81]:
# 1,2 保证 [[3,3,3],[4,4,4]] 
# 0,1 保证 [3,3,3]
# 0,3 保证 [3,3,3]
sess.run(tf.strided_slice(input, [1, 0, 0], [2, 1, 3], [1, 1, 1]))

array([[[3, 3, 3]]])

In [82]:
# 1,2 保证 [[3,3,3],[4,4,4]] 
# 0,2 保证 [[3,3,3],[4,4,4]] 
# 0,3 保证 [[3,3,3],[4,4,4]] 
sess.run(tf.strided_slice(input, [1, 0, 0], [2, 2, 3], [1, 1, 1]))

array([[[3, 3, 3],
        [4, 4, 4]]])

In [83]:
# 1,2 保证 [[3,3,3],[4,4,4]] 
# -1,-3 保证 [[4,4,4],[3,3,3]] 
# 0,3 保证 [[4,4,4],[3,3,3]] 
sess.run(tf.strided_slice(input, [1, -1, 0], [2, -3, 3], [1, -1, 1]))

array([[[4, 4, 4],
        [3, 3, 3]]])

## reader 输出结果

In [2]:
Py3 = sys.version_info[0] == 3

def _read_words(filename):
  with tf.gfile.GFile(filename, "r") as f:
    if Py3:
      return f.read().replace("\n", "<eos>").split()
    else:
      return f.read().decode("utf-8").replace("\n", "<eos>").split()


def _build_vocab(filename):
  data = _read_words(filename)

  counter = collections.Counter(data)
  count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

  words, _ = list(zip(*count_pairs))
  word_to_id = dict(zip(words, range(len(words))))

  return word_to_id


def _file_to_word_ids(filename, word_to_id):
  data = _read_words(filename)
  return [word_to_id[word] for word in data if word in word_to_id]


def ptb_raw_data(data_path=None):
  train_path = os.path.join(data_path, "ptb.train.txt")
  valid_path = os.path.join(data_path, "ptb.valid.txt")
  test_path = os.path.join(data_path, "ptb.test.txt")

  word_to_id = _build_vocab(train_path)
  train_data = _file_to_word_ids(train_path, word_to_id)
  valid_data = _file_to_word_ids(valid_path, word_to_id)
  test_data = _file_to_word_ids(test_path, word_to_id)
  vocabulary = len(word_to_id)
  return train_data, valid_data, test_data, vocabulary


def ptb_producer(raw_data, batch_size, num_steps, name=None):
  with tf.name_scope(name, "PTBProducer", [raw_data, batch_size, num_steps]):
    raw_data = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.int32)

    data_len = tf.size(raw_data)
    batch_len = data_len // batch_size
    data = tf.reshape(raw_data[0 : batch_size * batch_len],
                      [batch_size, batch_len])

    epoch_size = (batch_len - 1) // num_steps
    assertion = tf.assert_positive(
        epoch_size,
        message="epoch_size == 0, decrease batch_size or num_steps")
    with tf.control_dependencies([assertion]):
      epoch_size = tf.identity(epoch_size, name="epoch_size")

    i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
    x = tf.strided_slice(data, [0, i * num_steps],
                         [batch_size, (i + 1) * num_steps])
    x.set_shape([batch_size, num_steps])
    y = tf.strided_slice(data, [0, i * num_steps + 1],
                         [batch_size, (i + 1) * num_steps + 1])
    y.set_shape([batch_size, num_steps])
    return x, y

In [3]:
raw_data = ptb_raw_data("./ptb/data/")

In [4]:
train_data, valid_data, test_data, _ = raw_data

In [5]:
len(train_data), len(valid_data), len(test_data)

(929589, 73760, 82430)

In [6]:
word2id = _build_vocab('./ptb/data/ptb.train.txt')
id2word = dict(zip(word2id.values(), word2id.keys()))

In [7]:
" ".join([id2word[w] for w in train_data[100:120]])

'workers exposed to it more than N years ago researchers reported <eos> the asbestos fiber <unk> is unusually <unk> once'

In [108]:
batch_size = 20 # batch 的 大小
num_steps = 35 # time_step 35，每个 batch 中，每行数据的长度
epoch_size = ((len(train_data) // batch_size) - 1) // num_steps # 多少个 batch，一个 epoch
print("一共多少个 batch：", epoch_size)

一共多少个 batch： 1327


In [9]:
input_data, targets = ptb_producer(train_data, batch_size, num_steps, name=None)

In [10]:
# 20 个 batch 的矩阵数据，每个 batch 35 维，跑 1327 次
# 有一点反直觉。。。
input_data

<tf.Tensor 'PTBProducer/StridedSlice:0' shape=(20, 35) dtype=int32>

In [100]:
yam_train = "The core of the model consists of an LSTM cell that processes one word at a time \
and computes probabilities of the possible values for the next word in the sentence. \
The memory state of the network is initialized with a vector of zeros and gets updated after \
reading each word. For computational reasons, we will process data in mini batches of size batch size. \
In this example, it is important to note that current batch of words does not correspond to \
a sentence of words. Every word in a batch should correspond to a time t. TensorFlow will \
automatically sum the gradients of each batch for you.".split()

In [101]:
batch_size = 8
num_steps = 5
data_len = len(yam_train)
batch_len = data_len // batch_size
data = tf.reshape(yam_train[0 : batch_size * batch_len],
                  [batch_size, batch_len])

epoch_size = (batch_len - 1) // num_steps

In [102]:
batch_len

13

In [76]:
data

<tf.Tensor 'Reshape_5:0' shape=(8, 13) dtype=string>

In [95]:
x = tf.strided_slice(data, [0, i * num_steps], [batch_size, (i + 1) * num_steps])
x.set_shape([batch_size, num_steps])
y = tf.strided_slice(data, [0, i * num_steps + 1],[batch_size, (i + 1) * num_steps + 1])
y.set_shape([batch_size, num_steps])

In [91]:
# i = 0
sess.run(x)

array([[b'The', b'core', b'of', b'the', b'model'],
       [b'word', b'at', b'a', b'time', b'and'],
       [b'next', b'word', b'in', b'the', b'sentence.'],
       [b'with', b'a', b'vector', b'of', b'zeros'],
       [b'computational', b'reasons,', b'we', b'will', b'process'],
       [b'In', b'this', b'example,', b'it', b'is'],
       [b'does', b'not', b'correspond', b'to', b'a'],
       [b'should', b'correspond', b'to', b'a', b'time']], dtype=object)

In [110]:
# i = 1
sess.run(x)

array([[b'consists', b'of', b'an', b'LSTM', b'cell'],
       [b'computes', b'probabilities', b'of', b'the', b'possible'],
       [b'The', b'memory', b'state', b'of', b'the'],
       [b'and', b'gets', b'updated', b'after', b'reading'],
       [b'data', b'in', b'mini', b'batches', b'of'],
       [b'important', b'to', b'note', b'that', b'current'],
       [b'sentence', b'of', b'words.', b'Every', b'word'],
       [b't.', b'TensorFlow', b'will', b'automatically', b'sum']],
      dtype=object)

# ptb-word-lm

In [21]:
vocab_size = 10000
size = hidden_size = 128

In [35]:
with tf.variable_scope("embedding", reuse=True):
    embedding = tf.get_variable("embedding", [vocab_size, size], dtype=tf.float16)
inputs = tf.nn.embedding_lookup(embedding, input_data)

In [36]:
inputs

<tf.Tensor 'embedding_lookup_1:0' shape=(20, 35, 128) dtype=float16>

In [2]:
sess = tf.Session()

## dropout

In [12]:
X = np.array([[2.,3.],[4.,5.],[6.,7.]])
X

array([[2., 3.],
       [4., 5.],
       [6., 7.]])

In [14]:
sess.run(tf.nn.dropout(tf.convert_to_tensor(X), keep_prob=1.0))

array([[2., 3.],
       [4., 5.],
       [6., 7.]])

In [21]:
sess.run(tf.nn.dropout(tf.convert_to_tensor(X), keep_prob=0.9))

array([[2.22222222, 3.33333333],
       [4.44444444, 5.55555556],
       [6.66666667, 7.77777778]])

In [32]:
sess.run(tf.nn.dropout(tf.convert_to_tensor(X), keep_prob=0.8))

array([[2.5 , 3.75],
       [5.  , 0.  ],
       [0.  , 8.75]])

In [34]:
4*1/0.8

5.0